In [ ]:
!pip install -U stanza conllu==4.5 tqdm

In [6]:
!wget -O eval.py https://raw.githubusercontent.com/UniversalDependencies/tools/refs/heads/master/eval.py
!chmod +x eval.py

--2025-05-13 14:14:24--  https://raw.githubusercontent.com/UniversalDependencies/tools/refs/heads/master/eval.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 43784 (43K) [text/plain]
Saving to: ‘eval.py’

eval.py             100%[===================>]  42.76K  --.-KB/s    in 0.01s   

2025-05-13 14:14:24 (3.23 MB/s) - ‘eval.py’ saved [43784/43784]



In [3]:
import stanza, pathlib
stanza.download('sv')

INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Downloading default packages for language: sv (Swedish) ...


INFO:stanza:Downloaded file to /root/stanza_resources/sv/default.zip
INFO:stanza:Finished downloading models and saved to /root/stanza_resources


In [8]:
!wget -O sv_talbanken.tar.gz https://github.com/UniversalDependencies/UD_Swedish-Talbanken/archive/refs/tags/r2.16.tar.gz
!tar -xf sv_talbanken.tar.gz

--2025-05-13 14:14:51--  https://github.com/UniversalDependencies/UD_Swedish-Talbanken/archive/refs/tags/r2.16.tar.gz
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/UniversalDependencies/UD_Swedish-Talbanken/tar.gz/refs/tags/r2.16 [following]
--2025-05-13 14:14:51--  https://codeload.github.com/UniversalDependencies/UD_Swedish-Talbanken/tar.gz/refs/tags/r2.16
Resolving codeload.github.com (codeload.github.com)... 140.82.113.10
Connecting to codeload.github.com (codeload.github.com)|140.82.113.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/x-gzip]
Saving to: ‘sv_talbanken.tar.gz’

sv_talbanken.tar.gz     [    <=>             ]   4.33M  6.76MB/s    in 0.6s    

2025-05-13 14:14:52 (6.76 MB/s) - ‘sv_talbanken.tar.gz’ saved [4542701]



In [17]:
TB="UD_Swedish-Talbanken-r2.16"

TEST=f"{TB}/sv_talbanken-ud-test.conllu"
DEV=f"{TB}/sv_talbanken-ud-dev.conllu"
TRAIN=f"{TB}/sv_talbanken-ud-train.conllu"

In [21]:
def gold_token_texts(sentence):
    """
    Yield the surface‑form tokens in a UD sentence produced by
    CoNLL.conll2dict(), skipping MWT shells (id with “‑”) and
    empty nodes (id with “.”).
    """
    for item in sentence:
        for tok in (item if isinstance(item, list) else [item]):
            tid = str(tok["id"])
            if "-" in tid or "." in tid:
                continue
            yield tok["text"]

In [1]:
from stanza.utils.conll import CoNLL
from tqdm import tqdm
import stanza, pathlib

nlp = stanza.Pipeline(
        lang='sv',
        processors='tokenize,pos,lemma,depparse',
        tokenize_pretokenized=True,
        use_gpu=True)

INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Loading these models for language: sv (Swedish):
| Processor | Package            |
----------------------------------
| tokenize  | talbanken          |
| pos       | talbanken_charlm   |
| lemma     | talbanken_nocharlm |
| depparse  | talbanken_charlm   |

INFO:stanza:Using device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Loading: depparse
INFO:stanza:Done loading processors!


In [ ]:
from conllu import parse_incr

for split in ["test", "dev", "train"]:
    INFILE=f"UD_Swedish-Talbanken-r2.16/sv_talbanken-ud-{split}.conllu"
    OUTFILE = f"talbanken-stanza-{split}.conllu"

    with open(INFILE, encoding="utf-8") as gold,\
        open(OUTFILE,  "w", encoding="utf-8") as out:

        for tokenlist in tqdm(parse_incr(gold)):
            doc = nlp(tokenlist.metadata["text"])

            lines = "{:C}".format(doc).split("\n")
            lines = [line for line in lines if not line.startswith("#")]

            mdlines = []
            for mditem in tokenlist.metadata:
                mdlines.append(f"# {mditem}: {tokenlist.metadata[mditem]}")
            lines = mdlines + lines
            out.write("\n".join(lines))
            out.write("\n\n")


1219it [14:30,  1.40it/s]
504it [06:48,  1.23it/s]
1189it [11:40,  1.99it/s]